In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix, csr_matrix, coo_matrix
from scipy.sparse.linalg import aslinearoperator

np.set_printoptions(linewidth=100000)
np.set_printoptions(precision=5)

from pyiga import bspline, assemble, vform, geometry, vis, solvers, topology, bspline_cy


In [4]:
kv = bspline.KnotVector(np.array([0.0,0.0,0.0,0.25,0.5,0.75,1.0,1.0,1.0], dtype=np.float64),2)

In [6]:
kv

KnotVector(array([0.  , 0.  , 0.  , 0.25, 0.5 , 0.75, 1.  , 1.  , 1.  ]), 2)

In [7]:
a = kv.greville()

In [9]:
kv = bspline.make_knots(2,0.0,1.0,30, mult=2)

In [10]:
print(kv)

<KnotVector p=2 sz=64>


In [11]:
kv.findspan(0.5)

32